In [1]:
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 

plt.rcParams['figure.figsize'] = [30, 10]

In [2]:
def save_slices(img):
    h1 = cv2.hconcat([img[:,:,0],img[:,:,1],img[:,:,2],img[:,:,3]])
    h2 = cv2.hconcat([img[:,:,4],img[:,:,5],img[:,:,6],img[:,:,7]])
    return cv2.vconcat([h1,h2])


def get_bin(i):
    binary = bin(i).replace('0b','')
    while len(binary) < 8:
        binary = '0' + binary
    return binary

def bitplane_slice(image:np.array):
    img = image.copy()
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    h,w = img.shape
    slices = np.zeros(shape=(h,w,8))
    for row in range(h):
        for col in range(w):
            b = get_bin(img[row,col])
            for index in range(8):
                slices[row,col,index] = int(b[index])
    
    return slices

def xor_slices (img1,img2):
    return np.array(np.logical_xor(img1,img2),dtype=np.uint8)

def get_moving_objs(img):
    h,w,_ = img.shape
    result = np.zeros(shape=(h,w))
    for row in range(h):
        for col in range(w):
            tmp = 0 
            for index in range(7,3,-1):
                tmp += 2**index * img[row,col,7-index]
            if tmp >= 80 :
                result[row,col] = tmp 
    return result.astype(np.uint8)

def enhancement(img):
    denoise = cv2.fastNlMeansDenoising(img,None,32,32,12)
    ret,thresh = cv2.threshold(denoise,28,127,cv2.THRESH_BINARY)
    return denoise,thresh
    


In [3]:
for i in range(3):
    img1 = cv2.imread("./inputs/P2/football_f{0}_01.png".format(i+1))
    img2 = cv2.imread("./inputs/P2/football_f{0}_02.png".format(i+1))
    
    img1_b = bitplane_slice(img1)
    img2_b = bitplane_slice(img2)
    slices_1 = cv2.threshold(save_slices(img1_b), 0.5, 255, cv2.THRESH_BINARY)[1]
    slices_2 = cv2.threshold(save_slices(img2_b), 0.5, 255, cv2.THRESH_BINARY)[1]
    cv2.imwrite("./outputs/P02/bitplane-slices-f{0}-frame-1.png".format(i+1),slices_1)
    cv2.imwrite("./outputs/P02/bitplane-slices-f{0}-frame-2.png".format(i+1),slices_2)

    xor_ = xor_slices(img1_b,img2_b)
    xor_save = save_slices(xor_)
    print(xor_save.shape)
    img_binary = cv2.threshold(xor_save, 0.5, 255, cv2.THRESH_BINARY)[1]
    cv2.imwrite("./outputs/P02/bitplane-slices-f{0}-xor.png".format(i+1),img_binary)
    
    objs = get_moving_objs(xor_)
    denoise,thresh = enhancement(objs)
    
    plt.subplot(1,3,1)
    plt.imshow(objs,cmap="gray")
    plt.axis("off")
    plt.title("Obtained by 4 higher bit-planes")
    plt.subplot(1,3,2)
    plt.imshow(denoise,cmap="gray")
    plt.axis("off")
    plt.title("Denoised by Mean filter")
    plt.subplot(1,3,3)
    plt.imshow(thresh,cmap="gray")
    plt.axis("off")
    plt.title("Threshold on denoised result")
    plt.savefig("./outputs/P02/moving-objs-f{0}.png".format(i+1))
    plt.clf()
            
            
    

(1920, 5120)
(1920, 5120)
(1920, 5120)


<Figure size 2160x720 with 0 Axes>